In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver
from webdrivermanager.chrome import ChromeDriverManager

In [2]:
ChromeDriverManager.download_and_install
ChromeDriverManager.get_latest_version

<function webdrivermanager.chrome.ChromeDriverManager.get_latest_version(self)>

In [3]:
#chrome driver address
driver = webdriver.Chrome(r"C:\Users\TF2020\AppData\Local\rasjani\WebDriverManager\bin\chromedriver.exe")
driver_2 = webdriver.Chrome(r"C:\Users\TF2020\AppData\Local\rasjani\WebDriverManager\bin\chromedriver.exe")

function

In [128]:
def get_url(search):
    url_temp = 'https://www.amazon.com/s?k={}'
    search = search.replace(' ', '+')
    url = url_temp.format(search)
    url += '&page={}'
    return url


def item_stats(item):

    atag = item.h2.a
    description = atag.text.strip()
    ASIN = item.get('data-asin')
    item_url = 'https://www.amazon.com/dp/'+ ASIN
    
    price_parent = item.find('span',  'a-price')
    try:
        price = price_parent.find('span', 'a-offscreen').text
    except:
        price = 'no_price'
    try:
        rating = item.i.text.split(' ')[0]
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except:
        rating = 'no_rating'
        review_count = 'no_review'
    
    if item.find('i', {'aria-label': 'Amazon Prime'}):
            prime = 'Yes'
    else:
        prime = 'No'
    
    try: 
        promo = item.find('span', {'class': 'a-color-secondary'}).text 
        if promo == 'Featured from our brands' or promo == 'Sponsored' :
            is_promotion = promo
        else: is_promotion = 'No'
    except:
        is_promotion = 'No'

    



    stats = (description,ASIN,item_url,price,rating,review_count,prime,is_promotion)

    return stats

def seach_items(search):

    records = []
    url = get_url(search)

# range(1,21), I only need first 3 pages so
    for page in range(1,4):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'})

        for item in results:
            result = item_stats(item)
            records.append(result)
    return records

In [131]:
results = seach_items('TV')
df_amzn = pd.DataFrame(results)

In [105]:
url = get_url('TV')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
results = soup.find_all('div',{'data-component-type': 's-search-result'})
item = results[1]
item.find('span', {'class': 'a-color-secondary'}).text


'Sponsored'

In [93]:
item.find('div', {'data-asin': 'B08Z1RN7NP'})

In [112]:
atag = item.h2.a
description = atag.text.strip()
url = 'https://www.amazon.com'+atag.get('href')
item.get('data-asin')

'B094MZL6R1'

In [ ]:
#scraping the item deeper info
driver_2.get(item_url)
soup_item = BeautifulSoup(driver_2.page_source, 'html.parser')
try:
    item_result = soup_item.find('div',{'id': 'prodDetails'},{'class': 'a-section'})
    date_parent = item_result.find_all('td',{'class':'a-size-base prodDetAttrValue'})
    Release_date = date_parent[-1].text
except:
    Release_date = 'Unknown'